In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import importlib

module_path = '/Users/morton/PycharmProjects/psifr'
if module_path not in sys.path:
    sys.path.append(module_path)
from psifr import fr
data_file = os.path.join(module_path, 'data', 'cfr_raw_data.csv')

In [3]:
df = pd.read_csv(data_file, dtype={'category': 'category'})
df.category.cat.as_ordered(inplace=True)

study = df.query('trial_type == "study"').copy()
recall = df.query('trial_type == "recall"').copy()
data = fr.merge_lists(study, recall, 
                      list_keys=['list_type', 'list_category'], 
                      study_keys=['category'])

In [3]:
list1 = data.loc[(data['subject'] == 1) & (data['list'] == 37)]
rec1 = list1.query('repeat == 0').sort_values('output')
rec1

,subject,list,item,input,output,recalled,repeat,intrusion,list_type,list_category,category
903,1,37,WATCH,24.0,1.0,True,0,False,pure,obj,obj
901,1,37,SKATEBOARD,22.0,2.0,True,0,False,pure,obj,obj
897,1,37,KNIFE,18.0,3.0,True,0,False,pure,obj,obj
896,1,37,TROPHY,17.0,4.0,True,0,False,pure,obj,obj
902,1,37,VASE,23.0,5.0,True,0,False,pure,obj,obj
889,1,37,DRESSER,10.0,6.0,True,0,False,pure,obj,obj
880,1,37,LAWN MOWER,1.0,7.0,True,0,False,pure,obj,obj
898,1,37,INTERCOM,19.0,8.0,True,0,False,pure,obj,obj
890,1,37,MODEM,11.0,9.0,True,0,False,pure,obj,obj
894,1,37,BARCODE SCANNER,15.0,10.0,True,0,False,pure,obj,obj


In [24]:
outputs = rec1['input'].to_numpy()
n_recall = rec1['output'].max()
from_mask = ((rec1['repeat'] == 0) & ~rec1['intrusion']).to_numpy()
to_mask = from_mask
op = rec1['output'].to_numpy()
category = rec1['category'].to_numpy()

In [23]:
outputs

array([24., 22., 18., 17., 23., 10.,  1., 19., 11., 15.,  7., 16., 20.,
       12., nan,  2.,  3.,  4.,  5.,  6.,  8.,  9., 13., 14., 21.])

In [32]:
import importlib
import psifr.transitions as tr
importlib.reload(tr)
for prev, curr, poss in tr.transitions_masker(outputs, n_recall, from_mask, to_mask,
                                              category, lambda x, y: x == y):
    print(curr - prev)
    print(poss - prev)

-2.0
[ -2.  -6.  -7.  -1. -14. -23.  -5. -13.  -9. -17.  -8.  -4. -12. -22.
 -21. -20. -19. -18. -16. -15. -11. -10.  -3.]
-4.0
[ -4.  -5.   1. -12. -21.  -3. -11.  -7. -15.  -6.  -2. -10. -20. -19.
 -18. -17. -16. -14. -13.  -9.  -8.  -1.]
-1.0
[ -1.   5.  -8. -17.   1.  -7.  -3. -11.  -2.   2.  -6. -16. -15. -14.
 -13. -12. -10.  -9.  -5.  -4.   3.]
6.0
[  6.  -7. -16.   2.  -6.  -2. -10.  -1.   3.  -5. -15. -14. -13. -12.
 -11.  -9.  -8.  -4.  -3.   4.]
-13.0
[-13. -22.  -4. -12.  -8. -16.  -7.  -3. -11. -21. -20. -19. -18. -17.
 -15. -14. -10.  -9.  -2.]
-9.0
[-9.  9.  1.  5. -3.  6. 10.  2. -8. -7. -6. -5. -4. -2. -1.  3.  4. 11.]
18.0
[18. 10. 14.  6. 15. 19. 11.  1.  2.  3.  4.  5.  7.  8. 12. 13. 20.]
-8.0
[ -8.  -4. -12.  -3.   1.  -7. -17. -16. -15. -14. -13. -11. -10.  -6.
  -5.   2.]
4.0
[ 4. -4.  5.  9.  1. -9. -8. -7. -6. -5. -3. -2.  2.  3. 10.]
-8.0
[ -8.   1.   5.  -3. -13. -12. -11. -10.  -9.  -7.  -6.  -2.  -1.   6.]
9.0
[ 9. 13.  5. -5. -4. -3. -2. -1.  1.  2.  6.  

In [33]:
outputs

array([24., 22., 18., 17., 23., 10.,  1., 19., 11., 15.,  7., 16., 20.,
       12., nan,  2.,  3.,  4.,  5.,  6.,  8.,  9., 13., 14., 21.])

In [4]:
subj = data.query("subject == 1 and list_type == 'mixed'")
#recalls = [rec['input'].to_numpy() for name, rec in subj.groupby('list')]
#recalls
subj

,subject,list,item,input,output,recalled,repeat,intrusion,list_type,list_category,category
26,1,2,SEAN PENN,1.0,5.0,True,0,False,mixed,mixed,cel
27,1,2,AUDREY HEPBURN,2.0,NaN,False,0,False,mixed,mixed,cel
28,1,2,ST PATRICKS CATHEDRAL,3.0,11.0,True,0,False,mixed,mixed,loc
29,1,2,LES INVALIDES,4.0,NaN,False,0,False,mixed,mixed,loc
30,1,2,GREAT ZIMBABWE RUINS,5.0,NaN,False,0,False,mixed,mixed,loc
...,...,...,...,...,...,...,...,...,...,...,...
1173,1,48,OAHU BEACH,21.0,7.0,True,0,False,mixed,mixed,loc
1174,1,48,GATEWAY ARCH,22.0,6.0,True,0,False,mixed,mixed,loc
1175,1,48,WHITE HOUSE,23.0,2.0,True,0,False,mixed,mixed,loc
1176,1,48,WRIGLEY BUILDING,24.0,1.0,True,0,False,mixed,mixed,loc


In [32]:
import importlib
import psifr.transitions as tr
importlib.reload(tr)

for prev, curr, poss in tr.transitions_masker(outputs, n_recall, from_mask, to_mask,
                                              category, lambda x, y: x == y):
    print(curr - prev)
    print(poss - prev)

-2.0
[ -2.  -6.  -7.  -1. -14. -23.  -5. -13.  -9. -17.  -8.  -4. -12. -22.
 -21. -20. -19. -18. -16. -15. -11. -10.  -3.]
-4.0
[ -4.  -5.   1. -12. -21.  -3. -11.  -7. -15.  -6.  -2. -10. -20. -19.
 -18. -17. -16. -14. -13.  -9.  -8.  -1.]
-1.0
[ -1.   5.  -8. -17.   1.  -7.  -3. -11.  -2.   2.  -6. -16. -15. -14.
 -13. -12. -10.  -9.  -5.  -4.   3.]
6.0
[  6.  -7. -16.   2.  -6.  -2. -10.  -1.   3.  -5. -15. -14. -13. -12.
 -11.  -9.  -8.  -4.  -3.   4.]
-13.0
[-13. -22.  -4. -12.  -8. -16.  -7.  -3. -11. -21. -20. -19. -18. -17.
 -15. -14. -10.  -9.  -2.]
-9.0
[-9.  9.  1.  5. -3.  6. 10.  2. -8. -7. -6. -5. -4. -2. -1.  3.  4. 11.]
18.0
[18. 10. 14.  6. 15. 19. 11.  1.  2.  3.  4.  5.  7.  8. 12. 13. 20.]
-8.0
[ -8.  -4. -12.  -3.   1.  -7. -17. -16. -15. -14. -13. -11. -10.  -6.
  -5.   2.]
4.0
[ 4. -4.  5.  9.  1. -9. -8. -7. -6. -5. -3. -2.  2.  3. 10.]
-8.0
[ -8.   1.   5.  -3. -13. -12. -11. -10.  -9.  -7.  -6.  -2.  -1.   6.]
9.0
[ 9. 13.  5. -5. -4. -3. -2. -1.  1.  2.  6.  

In [68]:
list1 = list1.query('repeat == 0').sort_values('output')

In [72]:
items = list1['input'].to_list()
recalls = list1.loc[list1['output'].notna(), 'input'].to_list()
for i in range(len(recalls)):
    print(recalls[i])
    print(recalls[i + 1])
    print(items[i + 1:])

24.0
22.0
[22.0, 18.0, 17.0, 23.0, 10.0, 1.0, 19.0, 11.0, 15.0, 7.0, 16.0, 20.0, 12.0, nan, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 13.0, 14.0, 21.0]
22.0
18.0
[18.0, 17.0, 23.0, 10.0, 1.0, 19.0, 11.0, 15.0, 7.0, 16.0, 20.0, 12.0, nan, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 13.0, 14.0, 21.0]
18.0
17.0
[17.0, 23.0, 10.0, 1.0, 19.0, 11.0, 15.0, 7.0, 16.0, 20.0, 12.0, nan, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 13.0, 14.0, 21.0]
17.0
23.0
[23.0, 10.0, 1.0, 19.0, 11.0, 15.0, 7.0, 16.0, 20.0, 12.0, nan, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 13.0, 14.0, 21.0]
23.0
10.0
[10.0, 1.0, 19.0, 11.0, 15.0, 7.0, 16.0, 20.0, 12.0, nan, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 13.0, 14.0, 21.0]
10.0
1.0
[1.0, 19.0, 11.0, 15.0, 7.0, 16.0, 20.0, 12.0, nan, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 13.0, 14.0, 21.0]
1.0
19.0
[19.0, 11.0, 15.0, 7.0, 16.0, 20.0, 12.0, nan, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 13.0, 14.0, 21.0]
19.0
11.0
[11.0, 15.0, 7.0, 16.0, 20.0, 12.0, nan, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 9.0, 13.0, 14.0, 21.0]
11.0
15.

IndexError: list index out of range

In [14]:
rec1 = list1.query('repeat == 0').sort_values('output')
n = 0
rec1.iloc[n]

subject              1
list                37
item             WATCH
input               24
output               1
recalled          True
repeat               0
intrusion        False
list_type         pure
list_category      obj
category           obj
Name: 903, dtype: object

In [25]:
rec1['output'].max()

15.0

In [46]:
import importlib
import psifr.transitions as tr
importlib.reload(tr)
rec1.loc[:, '_from_mask'] = rec1['input'].notna()
rec1.loc[:, '_to_mask'] = rec1['input'].notna()
list_length = 24
list_actual = []
list_possible = []
for prev, curr, poss in tr._transition_masker(rec1):
    list_actual.append(curr['input'] - prev['input'])
    list_possible.extend(np.subtract(poss['input'].to_numpy(), 
                                     prev['input']))
lags = np.arange(-list_length + 1, list_length + 1)
actual = pd.Series(np.histogram(list_actual, lags)[0], index=lags[:-1])
possible = pd.Series(np.histogram(list_possible, lags)[0], index=lags[:-1])

In [52]:
importlib.reload(tr)
subj = data.query('subject == 1')
subj.loc[:, '_from_mask'] = subj['input'].notna()
subj.loc[:, '_to_mask'] = subj['input'].notna()
rec = subj.query('repeat == 0').sort_values('output')
tr._subj_lag_crp(rec)

/Users/morton/PycharmProjects/psifr/psifr/transitions.py:79: RuntimeWarning: invalid value encountered in true_divide
  'prob': actual / possible}, index=lags[:-1])


,actual,possible,prob
-23.0,1,48,0.020833
-22.0,3,84,0.035714
-21.0,3,114,0.026316
-20.0,3,125,0.024000
-19.0,2,139,0.014388
-18.0,2,164,0.012195
-17.0,8,178,0.044944
-16.0,3,193,0.015544
-15.0,11,223,0.049327
-14.0,5,233,0.021459


In [56]:
rec = data.query('repeat == 0').sort_values(['subject', 'list', 'output'])
rec.loc[:, '_from_mask'] = rec['input'].notna()
rec.loc[:, '_to_mask'] = rec['input'].notna()

In [58]:
importlib.reload(tr)
for i, subj in rec.groupby('subject'):
    print(i)
    tr._subj_lag_crp(subj)

1
2
3
4
5
6
7
8
10
11
12
15
16
18
20
22
23
24
25
26


IndexError: single positional indexer is out-of-bounds